In [1]:
import sigpde.torch as sig
import torch
import math
from samplers import *
device = torch.device('cuda:0')

In [2]:
kernel = sig.SigPDE(sig.kernels.LinearKernel(), 1)
kernel2 = sig.RobustSigPDE(sig.kernels.LinearKernel(), 1)

In [3]:
x = generate(30, 512, 50, device=device)
y = generate(5, 512, 50, device=device)
z, w = sample(30)
z = add_time(z)
x = add_time(x)
w = time_norm(w)

In [4]:
kernel = sig.SigPDE(sig.kernels.LinearKernel(), 1)
kernel2 = sig.RobustSigPDE(sig.kernels.LinearKernel(), 1)

In [6]:
kernel2 = sig.RobustSigPDE(sig.kernels.LinearKernel(), 0)
kernel2.gram(x)

tensor([[1.9805, 0.8663, 1.0279, 0.8529, 0.8871, 1.0420, 1.0450, 1.0704, 1.0650,
         0.9543, 1.2051, 0.9703, 0.9700, 1.1463, 1.0921, 0.9167, 1.1679, 1.0561,
         0.9102, 0.9224, 1.0831, 1.0494, 1.2831, 0.8594, 1.0198, 0.9452, 0.9285,
         1.0361, 0.8914, 1.1245],
        [0.8663, 1.9795, 0.7941, 1.0538, 1.1110, 0.9309, 0.9063, 1.1107, 1.0248,
         1.1035, 0.8771, 1.0646, 0.9277, 1.0326, 1.0843, 1.1607, 0.9473, 1.1199,
         1.0949, 0.9466, 0.9155, 1.0332, 0.9218, 1.1000, 0.9902, 1.0378, 0.8063,
         1.0251, 1.2305, 1.0871],
        [1.0279, 0.7941, 1.9796, 1.0624, 1.1480, 0.9680, 1.0984, 0.8628, 0.8738,
         1.0490, 1.0472, 1.2080, 1.0071, 1.0277, 1.1192, 1.0228, 0.8540, 1.0146,
         0.9732, 1.0232, 1.0416, 0.9227, 1.0702, 1.1403, 0.9297, 1.1180, 1.1635,
         1.0943, 0.9807, 0.8747],
        [0.8529, 1.0538, 1.0624, 1.9808, 1.0007, 1.1018, 0.9523, 1.0505, 0.8515,
         0.9549, 0.9722, 0.9975, 0.9756, 1.1594, 1.0868, 1.2257, 0.9670, 1.1761,
       

In [124]:
z, w = sample(1)
z = add_time(z)
c = std_norm(kernel.pairwise(z))
ff = lambda s : kernel.pairwise(z, x_scale=s) - c

In [4]:
def bisection_single_eval(f, a, b, tol=1e-6, max_iter=100):
    fa = f(a)
    fb = f(b)
    k = 0
    if fa * fb >= 0:
        raise ValueError("Function values at the endpoints must have opposite signs.")
    
    for _ in range(max_iter):
        k += 1
        # Compute midpoint
        c = (a + b) / 2
        fc = f(c)

        # Update the interval and reuse function evaluations
        if fa * fc < 0:
            b, fb = c, fc  # Update the right endpoint
        else:
            a, fa = c, fc  # Update the left endpoint
            
        # Stopping criterion based on function value
        if abs(fc) < tol:
            print(k)
            return a, b

    raise RuntimeError("Maximum number of iterations reached without convergence.")

In [5]:
def itp_single_eval(f, a=0, b=1,tol=1e-6, max_iter=100):
    fa = f(a)
    fb = f(b)
    k = 0

    if fa * fb >= 0:
        raise ValueError("Function values at the endpoints must have opposite signs.")
    
    for _ in range(max_iter):
        k += 1
        # Midpoint and interpolation calculation
        mid = (a + b) / 2
        interp = (a * fb - b * fa) / (fb - fa)
        t = 0.2  # Blending parameter (adjust if necessary)
        x = (1 - t) * mid + t * interp

        fx = f(x)

        # Update the interval and reuse function evaluations
        if fx * fa < 0:
            b, fb = x, fx  # Update the right endpoint
        else:
            a, fa = x, fx  # Update the left endpoint
            
        if abs(fx) < tol:  # Stopping based on function value
            print(k)
            return a, b
            
    return (a + b) / 2, k

In [6]:
def secant_single_eval(f, x0, x1, tol=1e-6, max_iter=100):
    f0 = f(x0)
    f1 = f(x1)
    k = 0

    if abs(f0) < tol:
        return x0
    if abs(f1) < tol:
        return x1

    for _ in range(max_iter):
        k += 1
        # Compute the next point using the secant formula
        x2 = x1 - f1 * (x1 - x0) / (f1 - f0)
        f2 = f(x2)

        # Check stopping criteria
        if abs(f2) < tol:
            return x2, f2, k

        # Update for the next iteration
        x0, f0 = x1, f1
        x1, f1 = x2, f2

In [7]:
def itp_optimized(f, a, b, tol, max_iter, alpha=0.2, beta=1.0):
    # Initial evaluations
    f_a = f(a)
    f_b = f(b)
    k = 0
    if f_a * f_b > 0:
        raise ValueError("Function must have opposite signs at endpoints a and b.")

    for iteration in range(max_iter):
        k += 1
        # Compute midpoint and its function value (reuse evaluations when possible)
        m = (a + b) / 2
        f_m = f(m) if iteration == 0 else f_m  # First iteration requires f(m)

        # Interpolation
        delta = beta * (b - a) / 2
        g = m - (1 if f_m > 0 else -1) * min(abs(f_m) * delta, delta)

        # Truncation
        t = m + (g - m) * min(1, alpha * (b - a))

        # Evaluate f(t) only once
        f_t = f(t)

        # Update bounds and reuse evaluations
        if f_a * f_t < 0:
            b, f_b = t, f_t
        else:
            a, f_a = t, f_t
            
        # Check stopping criteria
        if f_t == 0 or abs(f_t) < tol:
            print(k)
            return a, b

        # Update f_m for reuse in the next iteration
        f_m = f_m if t == m else f(t)


In [8]:
def newton_raphson(f, x0, c, tol=1e-6, max_iter=100):
    k = 0
    
    for _ in range(max_iter):
        k += 1
        fx, dfx = f(x0)
        
        fx = fx - c
        
        if abs(fx) < tol:
            return x0, fx, k
        x0 = x0 - fx / dfx

In [50]:
def chandrupatla(f, x0, x1, tol=1e-6, maxit=100):
    b = x1
    a = x0
    c = x0
    fa = f(a)
    fb = f(b)
    fc = fa
    
    k = 0
    t = 0.5
    
    for _ in range(maxit):
        k += 1
        xt = a + t * (b - a)
        ft = f(xt)
        
        if ft * fa >= 0:
            c = a
            fc = fa
        else:
            c = b
            b = a
            fc = fb
            fb = fa
            
        a = xt
        fa = ft
        
        if abs(fa) < abs(fb):
            xm = a
            fm = fa
        else:
            xm = b
            fm = fb
            
        if abs(fm) < tol:
            print(k)
            return fm
                        
        xi = (a - b) / (c - b)
        phi = (fa - fb) / (fc - fb)
        
        if phi**2 < xi and (1 - phi)**2 < 1 - xi:
            t = fa / (fb - fa) * fc / (fb - fc) + (c - a) / (b - a) * fa / (fc - fa) * fb / (fc - fb)
        else:
            t = 0.5
    

In [51]:
x = generate(1, 512, 50, device=device)
c = std_norm(kernel.pairwise(x))
ff = lambda s : kernel.pairwise(x, x_scale=s) - c
a = torch.tensor([0], device=x.device, dtype=x.dtype)
b = torch.tensor([1], device=x.device, dtype=x.dtype)
chandrupatla(ff, x0=a, x1=b, tol=1e-8)

6


tensor([1.2433e-11], device='cuda:0', dtype=torch.float64)

In [40]:
a = torch.tensor([0], device=x.device, dtype=x.dtype)
b = torch.tensor([1], device=x.device, dtype=x.dtype)
a, b = bisection_single_eval(ff, a=a, b=b, tol=0.01)
b_secant, val_secant, k_secant = secant_single_eval(ff, a, b, tol=1e-7, max_iter=200)
print(b_secant)
print(val_secant)
print(k_secant)

6
tensor([0.6388], device='cuda:0', dtype=torch.float64)
tensor([-4.4771e-12], device='cuda:0', dtype=torch.float64)
3


In [33]:
a = torch.tensor([0], device=x.device, dtype=x.dtype)
b = torch.tensor([1], device=x.device, dtype=x.dtype)
#a, b = bisection_single_eval(ff, a=a, b=b, tol=0.01, max_iter=100)
a, b =  itp_single_eval(ff, a=a, b=b, tol=0.01)
b_secant, val_secant, k_secant = secant_single_eval(ff, a, b, tol=1e-7, max_iter=200)
print(b_secant)
print(val_secant)
print(k_secant)

25


In [28]:
a = torch.tensor([0], device=x.device, dtype=x.dtype)
b = torch.tensor([1], device=x.device, dtype=x.dtype)
a, b = itp_single_eval(ff, a=a, b=b, tol=0.01)
#a, b = itp_optimized(ff, a=a, b=b, tol=0.01, max_iter=100)
df = lambda s : kernel2.pairwise_norm(x, s)
b_nr, s_nr, k_nr = newton_raphson(df, (a + b) / 2, c, tol=1e-7, max_iter=200)
print(b_nr)
print(s_nr)
print(k_nr)

7


AttributeError: 'RobustSigPDE' object has no attribute 'pairwise_norm'

In [68]:
print(kernel.pairwise(x, x_scale = b_secant) - c)
print(kernel.pairwise(x, x_scale = b_nr) - c)

tensor([1.7421e-10], device='cuda:0', dtype=torch.float64)
tensor([-6.3408e-11], device='cuda:0', dtype=torch.float64)
